In [47]:
import sys
import requests
from time import sleep

from tqdm import tqdm
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import gensim

In [48]:
sys.path.append('../')

In [49]:
from src import settings

In [50]:
client = MongoClient(settings.MONGODB_SETTINGS['host'])
db = client[settings.MONGODB_SETTINGS['db']]
col = db[settings.MONGODB_SETTINGS['collection']]

In [51]:
from src.models import make_texts_corpus
from src.utils import editorJs_data_to_text
import json
from gensim.matutils import cossim
from gensim import similarities

In [52]:
id2word = gensim.corpora.Dictionary.load(settings.PATH_DICTIONARY)
corpus = gensim.corpora.MmCorpus('../models/corpus.mm')
lda_model = gensim.models.ldamodel.LdaModel.load('../models/LDA.model')

INFO : loading Dictionary object from ../models/id2word.dictionary
INFO : Dictionary lifecycle event {'fname': '../models/id2word.dictionary', 'datetime': '2023-01-01T08:27:00.441545', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}
INFO : loaded corpus index from ../models/corpus.mm.index
INFO : initializing cython corpus reader from ../models/corpus.mm
INFO : accepted corpus with 177 documents, 4087 features, 36185 non-zero entries
INFO : loading LdaModel object from ../models/LDA.model
INFO : loading expElogbeta from ../models/LDA.model.expElogbeta.npy with mmap=None
INFO : setting ignored attribute state to None
INFO : setting ignored attribute dispatcher to None
INFO : setting ignored attribute id2word to None
INFO : LdaModel lifecycle event {'fname': '../models/LDA.model', 'datetime': '2023-01-01T08:27:00.512557', 'gensim': '4.2.0', 'python': '3.9.6 (tags/v3.

In [53]:
index = similarities.MatrixSimilarity(lda_model[corpus])

INFO : creating matrix with 177 documents and 57 features


In [54]:
for post in col.find():
    print(post['title'], 'idrs: ', post['idrs'] if 'idrs' in post else False)

ĐỒ ÁN THIẾT KẾ WEBSITE BLOG SITE idrs:  0
Regular Expression trong Javascript idrs:  1
Các bài tập java cơ bản idrs:  2
Những câu hỏi lập trình khi pv (2): bài toán ATM idrs:  3
Ví dụ sắp xếp nổi bọt với Java idrs:  4
Chương trình lọc thư rác bằng Java, code ví dụ idrs:  5
Đọc ghi file Excel trong Java sử dụng Apache POI idrs:  6
THREAD AND THREAD POOLS IN JAVA idrs:  7
Sử dụng CountDownLatch trong Java idrs:  8
Multithreading: Java Synchronized Blocks idrs:  9
Understand synchronized and improve lock performance in Java idrs:  10
Front-End Web Developers idrs:  11
Material Design for Web Design idrs:  12
Những kỹ năng cần phải có của một Web Designer idrs:  13
Rest API - cách Frontend và Backend nói chuyện với nhau qua ví dụ cụ thể idrs:  14
Điều cần chú ý để xây dựng RESTful API idrs:  15
Dựng một API RESTful đơn giản sử dụng Express và MongoDB idrs:  16
xây dựng API với Napa idrs:  17
Trí tuệ nhân tạo với Swift idrs:  18
Lợi ích và nguy cơ của trí tuệ nhân tạo (AI) idrs:  19
Liệu ch

In [55]:
# main_post = col.find_one({"_id": ObjectId("6388c04363cdee37de74f09c")})
main_post = col.find_one({"idrs": 165})
main_post['title']

'Sắc màu văn hóa các dân tộc vùng Tây Bắc'

In [56]:
content = editorJs_data_to_text(json.loads(main_post["content"]))
text_corpus = make_texts_corpus([content])
bow = id2word.doc2bow(next(text_corpus))

In [57]:
vector_doc = lda_model[bow]

In [58]:
sims = index[vector_doc]

In [59]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])
#for doc_position, doc_score in sims:
#    print(doc_score, doc_position)

In [60]:
print(main_post['title'])
i=0
for idrs, distance in sims:
    article = col.find_one({"idrs": idrs})
    print(i, idrs, distance, article['title'])
    i+=1
    if(i==100): break
    

Sắc màu văn hóa các dân tộc vùng Tây Bắc
0 166 0.99999994 Reactjs Là Gì? Những Điều Bạn Chưa Biết Về Reactjs.
1 165 0.93411946 Sắc màu văn hóa các dân tộc vùng Tây Bắc
2 110 0.8710784 “SỐNG XANH” CÓ KHÓ NHƯ NHIỀU NGƯỜI NGHĨ?
3 163 0.85975945 Ý nghĩa tà áo dài truyền thống Việt Nam – giá trị qua các thời kỳ lịch sử
4 162 0.8106241 Áo dài - nét đẹp đặc sắc của văn hoá Việt Nam
5 161 0.80356514 Lịch sử phát triển áo dài Việt Nam qua các thời kỳ
6 164 0.7463762 Văn hóa Tây Bắc - những mạch nguồn chảy mãi
7 29 0.7132419 Kinh nghiệm du lịch Hạ Long tự túc cho người lần đầu mới đi
8 107 0.66847634 Phong tục truyền thống trong ngày Tết cổ truyền của người Việt
9 109 0.66062427 Chuẩn bị đi du lịch? Cẩm nang từ A-Z giúp bạn có kỳ nghỉ TRỌN VẸN
10 30 0.5828639 Muốn khởi nghiệp? Hãy dành 5 phút đọc bài viết này.
11 108 0.477423 Giữ gìn, phát huy nét đẹp văn hóa Tết cổ truyền của Dân tộc Việt Nam
12 112 0.4298607 Kinh nghiệm chuẩn bị cho chuyến du lịch hoàn hảo
13 159 0.42216477 Cơ cấu tổ chức mới 

In [61]:
arr = [tup[0] for tup in sims[:10]]
if 'idrs' in main_post: 
    if main_post['idrs'] in arr: arr.remove(main_post['idrs'])
print(main_post['idrs'])
list(arr)

165


[166, 110, 163, 162, 161, 164, 29, 107, 109]